In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/myenv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/myenv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 16:49:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [3]:
sc = spark.sparkContext

**Zadanie 1*  
Na zbiorze danych '_Recipe Reviews ..._' wykonaj:

In [4]:
from pyspark.sql.functions import desc

df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



  
1.1  Zmień nazwę pierwszej kolumny z `_c0` na `id`. 

In [5]:
df_reviews = df_reviews.withColumnRenamed('_c0','id')
df_reviews.printSchema()

root
 |-- id: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



1.2  Wyświetl 10 najwyższych wartości w kolumnie `reply_count`.  

In [6]:
df_reviews.select(df_reviews.reply_count).sort(desc('reply_count')).show(10)

+-----------+
|reply_count|
+-----------+
|          3|
|          3|
|          3|
|          3|
|          3|
|          3|
|          2|
|          2|
|          2|
|          2|
+-----------+
only showing top 10 rows



1.3  Wyświetl 10 najwyższych sum wartości w kolumnie `best_score` dla każdego przepisu (grupowanie).  

In [7]:
df_reviews.groupby('recipe_code').agg({'best_score': 'sum'}).sort(desc('sum(best_score)')).show(10)

+-----------+---------------+
|recipe_code|sum(best_score)|
+-----------+---------------+
|       2832|        98863.0|
|      14299|        85497.0|
|      17826|        64880.0|
|       3309|        64247.0|
|      21444|        60755.0|
|      32480|        59867.0|
|      12540|        59195.0|
|       2912|        54032.0|
|      42083|        51975.0|
|      19731|        47905.0|
+-----------+---------------+
only showing top 10 rows



1.4  Które 10 przepisów miało najwięcej komentarzy? 

In [8]:
df_reviews.groupby('recipe_code').agg({'comment_id': 'count'}).sort(desc('count(comment_id)')).show(10)

+-----------+-----------------+
|recipe_code|count(comment_id)|
+-----------+-----------------+
|       2832|              725|
|      14299|              654|
|       3309|              509|
|      42083|              421|
|      32480|              397|
|      21444|              395|
|      12540|              368|
|      17826|              338|
|       2912|              332|
|      19731|              324|
+-----------+-----------------+
only showing top 10 rows



1.5  Wyświetl rozkład wartości w kolumnie `stars`.  

In [9]:
df_reviews.select(df_reviews.stars).describe().show()

+-------+------------------+
|summary|             stars|
+-------+------------------+
|  count|             18182|
|   mean|  4.28880211197888|
| stddev|1.5447863581965313|
|    min|                 0|
|    max|                 5|
+-------+------------------+



**Zadanie 2**  
Wczytaj zbiór danych `employee` nakazując Sparkowi wywnioskowanie bardziej optymalnych typów danych niż domyślny typ `string`.

In [10]:
df = spark.read.csv('employee.csv', header=True, inferSchema=True)
df.printSchema()

[Stage 12:================================================>         (5 + 1) / 6]

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)



**Zadanie 3**  
Jaki jest czas wykonania operacji `df.filter(df["salary"] > 10000).count()` tym razem przy numerycznym typie kolumny `salary`? Jest jakaś różnica?

In [11]:
from datetime import datetime
import pyspark.sql.functions as F

df = spark.read.csv('employee.csv', header=True)

start = datetime.now()
df.filter(df["salary"] > 10000).count()
print(f'end: {datetime.now() - start}')

[Stage 14:================================================>         (5 + 1) / 6]

end: 0:00:14.515439


In [12]:
df = spark.read.csv('employee.csv', header=True)

df = df.withColumn("salary", F.col("salary").cast("decimal(10,2)"))
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [13]:
start = datetime.now()
df.filter(df["salary"] > 10000).count()
print(f'end: {datetime.now() - start}')

[Stage 18:================================================>         (5 + 1) / 6]

end: 0:00:16.627322


**Zadanie 4**  
Wykorzystując przykład z dokumentacji klasy `Bucketizer` (https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Bucketizer.html) podziel dane w kolumnie `age` zbioru `employee` na buckety co 10 lat (10-19, 20-29, ..., 60-69)
wyświetl te dane dla 20 pierwszych wierszy w formie surowej 
oraz całość grupując po bucketach i licząc ile osób znalazło się w każdym z nich.

In [14]:
from pyspark.sql.functions import col
from pyspark.ml.feature import Bucketizer

df = df.withColumn("age", F.col("age").cast("integer"))
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [15]:
splits = [10, 20, 30, 40, 50, 60, 70]
bucketizer = Bucketizer(splits=splits, inputCol="age", outputCol="age_bucket")

# Przekształcamy dane - dodajemy kolumnę z bucketami
bucketed_df = bucketizer.transform(df)

bucketed_df.select('*').show(20)

+---+----------+-------------------+---+-------+----------+
| id| firstname|           lastname|age| salary|age_bucket|
+---+----------+-------------------+---+-------+----------+
|  1|   Wisława|             Wlotka| 33|8303.97|       2.0|
|  2|     Agata|         Malinowski| 60|7960.70|       5.0|
|  3|Aleksandra|Brzęczyszczykiewicz| 25|8966.46|       1.0|
|  4|     Agata|               Glut| 55|7449.52|       4.0|
|  5|     Agata|              Pysla| 18|7788.84|       0.0|
|  6|     Marek|         Wróblewski| 26|8603.35|       1.0|
|  7|   Wisława|           Barański| 27|6830.76|       1.0|
|  8|     Marek|             Wlotka| 50|7255.81|       4.0|
|  9| Krzysztof|              Pysla| 58|6616.81|       4.0|
| 10|Aleksandra|         Malinowski| 60|6541.10|       5.0|
| 11|Mieczysław|           Barański| 52|7524.52|       4.0|
| 12| Katarzyna|              Pysla| 29|6680.68|       1.0|
| 13|      Adam|         Malinowski| 25|8819.53|       1.0|
| 14|Mieczysław|           Barański| 30|

In [19]:
bucketed_counts = (
    bucketed_df.groupBy("age_bucket")
    .count()
    .orderBy("age_bucket")
)
bucketed_counts.show()

[Stage 25:================================================>         (5 + 1) / 6]

+----------+-------+
|age_bucket|  count|
+----------+-------+
|       0.0| 784973|
|       1.0|3921202|
|       2.0|3921730|
|       3.0|3921517|
|       4.0|3921697|
|       5.0|3528881|
+----------+-------+

